# 1. Specify all parameters to be varied


In [1]:
from fedt import *

In [3]:
experiment = FEDTExperiment()

experiment.CAD_variables = [{
        'name' : 'side length',
        'data type' : 'ratio',
        'test_values' : [x for x in range(10, 20, 5)],
        'argname' : 'side length'
    }]
experiment.CAM_variables = [{
        'name' : 'layer height',
        'data type' : 'ratio',
        'test_values' : [x / 10.0 for x in range(5, 20, 5)], # i hate this so much, but we would need numpy arange to make it better
        'argname' : '--layer-height'
    },
    {
        'name' : 'fill pattern',
        'data type' : 'nominal',
        'test_values' : ['triangles', 'concentric'], # I would like them all to have the same named thing so I can call it
        'argname' : '--fill-pattern'
    }]

experiment.fab_repetitions = 2

sample_post_process_variable = {
    'name' : 'paint type',
    'data type' : 'nominal',
    'test_values' : ['lacquer', 'spraypaint'],
    'description' : 'use the {} on the object'
}
experiment.post_process_variables = []
experiment.post_process_repetitions = 1

sample_interaction_variable = {
    'name' : 'user push force level',
    'data type' : 'ratio',
    'test_values' : [x for x in range(10)],
    'instruction' : 'push on the button with force {} / 10' 
}
experiment.interaction_variables = [sample_interaction_variable]

experiment.measurement_variables = [{
        'name' : 'printed x-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed y-axis',
        'data type' : 'ratio'
    },
    {
        'name' : 'printed z-axis',
        'data type' : 'ratio'
    }
]
experiment.measurement_repetitions = 5

tea_hypothesis = (['material', 'measured z-axis' ], ['fill_pattern:concentric > triangle'])

###########

experiment.experiment_size()
experiment.label_all_conditions()

0


experiment.configure_for_freecad()
experiment.configure_for_printing()

This experiment will require fabricating 24 unique objects.
Users will perform 600 interactions.
360 total measurements will be recorded.


ModuleNotFoundError: No module named 'PySide2'

# 2. CAD

In [ ]:
# FreeCAD create cube    
def geometry_function(App, side_length='20 mm'):
    App.ActiveDocument.addObject("Part::Box","Box")
    App.ActiveDocument.ActiveObject.Label = "Cube"
    App.ActiveDocument.recompute()
    FreeCAD.ActiveDocument.getObject('Box').Width = side_length
    FreeCAD.ActiveDocument.getObject('Box').Length = side_length
    FreeCAD.ActiveDocument.getObject('Box').Height = side_length
    App.ActiveDocument.recompute()

def label_function(App, label_string="L0"):
    # https://wiki.freecad.org/Draft_ShapeString_tutorial
    ss=Draft.make_shapestring(String=label_string, FontFile=FONT_LOCATION, Size=5.0, Tracking=0.0)
    plm=FreeCAD.Placement()
    plm.Base=FreeCAD.Vector(4.0, 4.0, side=20)
    plm.Rotation.Q=(0.0, 0.0, 0, 1.0)
    ss.Placement=plm
    ss.Support=None
    Draft.autogroup(ss)
    FreeCAD.ActiveDocument.recompute()
    
    App.ActiveDocument.addObject('Part::Extrusion','Extrude')
    f = App.ActiveDocument.getObject('Extrude')
    f.Base = App.ActiveDocument.getObject('ShapeString')
    f.DirMode = "Normal"
    f.DirLink = None
    f.LengthFwd = 5.000000000000000
    f.LengthRev = 0.000000000000000
    f.Solid = False
    f.Reversed = False
    f.Symmetric = False
    f.TaperAngle = 0.000000000000000
    f.TaperAngleRev = 0.000000000000000
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('ShapeString').Visibility = False
    App.ActiveDocument.recompute()
    App.ActiveDocument.getObject('Extrude').Placement = App.Placement(App.Vector(2,2,0),App.Rotation(App.Vector(0,0,1),0))
    App.ActiveDocument.recompute()
    #FreeCAD.ActiveDocument.getObject('Extrude').Shape.exportStl('ext.stl')

experiment.configure_for_freecad()
experiment.label_function = label_function
experiment.geometry_function = geometry_function

# 3. CAM

In [ ]:
experiment.configure_for_printing()

# 4. Fabrication


In [ ]:
experiment.fabricate()

# 5. Post-Processing

In [ ]:
experiment.post_process()

# 6. User Interaction

In [ ]:
experiment.interact()

# 7. Measurement

In [ ]:
experiment.measure()

# 8. Tea-based hypothesis testing

In [ ]:
#from fedt_tea import do_experiment

'''tea_results = do_experiment(measurement_variables,
                            tea_hypothesis,
                            experiment_csv,
                            CAD_variables,
                            CAM_variables,
                            post_process_variables,
                            interaction_variables)'''

# 9. Reporting

In [5]:
experiment.report_latex()

KeyError: 'test_values'